There is a number of notable papers

Most grpahs are plotted on a log-log scale

## Maths of Power law
---

Exponential<br>
$f(x)=a^{x}$

General power law<br>
$f(x) = k \cdot x^\alpha$

Or shortly: f is proportional to x<br>
$f \propto x^\alpha$

On a Log-Log scale the expression<br>
$\log{f} = \log{k} + \alpha \cdot \log{x}$
<br>becomes a straight line

Fun fact: there are no axes on the Log-Log plots since logarithm is never zero (instead you infinitely converge towards zero along the axis)

Power law functions $k \cdot x^\alpha$ are defined by two parameters: 
- change dynamics $\alpha$
- scale factor $k$

If $\alpha < 1$ we say function is sublinear and it exposes the diminishing returns<br>
If $\alpha < 0$ the function is decreasing

On a Log-Log scale: 
- power function becomes a straight line 
- $\alpha$ defines the tangent of the straight line
- $k$ defines the bias

$\alpha$ is enough to evaluate the scaling rate of parameters. $k$ is reuired if we want to predict the concrete values

Here are the examples of different scaling factors - they help match available data points

<img src="img/powerlaw1.png" width=750>

And here we can see different powers - they help model the dependency kind

<img src="img/powerlaw2.png" width=750>

# Kaplan 2020
---

Authors from OpenAI explore two main characteristics of the model:
- N = model size (in parameters)
- D = training time (in tokens)

*N does not include embeddings as they are not very informative (not very controllable)

The task is to predict the behavior of model performance:
- L = performance <br>validation loss, aka cross-entropy of the corresponding language modeling task

Third important parameter
- C = compute (in FLOPs)

C is a combination of the two other parameters<br> $C = 6ND$. It optimization to a narrow set of points holding same compute

#### Why parameter C is importnant?
In practice C is often your only input and parameters N and D are under control

#### Where does 6ND come from?
Compute can be estimated as 6 FLOPs per one network weight = 1-2 on forward pass, 2 on backward, 2 on ...

Exact formulas also exist. For a standard Transformer model where L=input, d=dimensionality:<br>
$$\underset{projections}{3Ld^2} + \underset{attention}{L^2d}+\underset{output}{Ld^2} = 4Ld^2 + L^2d$$

But those are not as convenient, beacuase they depend on a particular architecture of the network, and we want our estimates to be as robust as possible

For a batch processing there is an alternative variant<br>$C=6NDB$






# Digest of Key findings
---
- __Performance depends strongly on scale, weakly on model shape__<br>Model performance increases with model scale (N,D,C) and does not depend much on the architecture<br><br>
- __Smooth power laws:__<br>Performance depends on scale parameters as power law $N^{-1}, D^{-1}, C^{-1}$, given that other parameters are set free<br><br>
- __Universality of overfitting__<br>this power law scaling is not additive, you have to scale both N and D simultaneously (ideal combination D = N^{0.75})<br><br>
- __Universality of training__<br>learning curves are universal and do not depend on N => one can use them to extrapolate <br>it contradicts an image<br><br>
- __Transfer improves with test performance__<br>distribution shift in data generates a constant penalty preserves the shape<br><br>
- __Sample efficiency__<br>arger models reach the same loss faster than small models<br><br>
- __Convergence is inefficient__<br><br>
- __Optimal batch size__<br><br>


# Main Laws
---

First, they explore individual laws (one argument)

### Model size
$$L(N) = L_0 + \bigg(\frac{N_c}{N}\bigg)^{0.076} \quad \text{where } N_c ∼ 8.8 × 10^{13}$$

Where did we get D?<br>
For each $N$ we train the model virtually "infinitely" $D \rightarrow \infty$. The limit that $L(N)$ converges to is assigned to each N<br>According to Kaplan there is always some $D^*$ training time after which loss function stabilizes on a plateau - the model "learned" everything it could have learned

Conclusion = The performance of a model of size N is distributed accoring to scaling law

The process is illustrated by the learning curves (see the limits on the right handside)

<img src="img/learning1.png" width=300>

Facts:
- Smaller models reach the plateau faster<br>there is less potential for learning<br><br>
- Larger models are always trained faster<br>given two models trained for the equal amount of time, the larger model will always outperform the smaller one<br><br>
- Final model performance (validational) = plateau height $L(N)$<br>defines how much we can potentially squeezer from thr model of that size<br><br>
- Model loss is diminishing according to power law presented above $L(N) \propto N^{-0.076}$<br><br>
- learning curves resemble sigmoids on a Log-Log scale<br>heating, active learning, plateau<br><br>

The irreducible loss bias $L_0$ is sometimes omited for simplicity

### Dataset size
D denotes the dataset size
$$L(D) = \bigg(\frac{Dc}{D}\bigg)^{0.095} \text{ where } D_c ∼ 5.4 × 10^{13}$$
For each value of $D$ we train the model (potentially multiple epochs) on this dataset until it converges

Conclusion = The performance that we can squeeze from the dataset of size D is according to power law

<img src="img/kaplan_d.png" width=400>

Note that here D is not a Training time! To get a training time refer to the Model size learning curves (plotted above)<br>And to get asymptotic training times refer to the yellow line from that graph

### Compute size

C is number of FLOPs

$$L(C) = \bigg(\frac{C_c}{C}\bigg)^{0.05} \text{where } C_c = 1.3 × 10^{22} \text{ FLOPs per hour}$$

For each compute $C$ check all $(N,D)$ pairs and assign corresponding loss values. One (N,D) pair is always optimal - it corresponds to minimal validation loss

*Loss is computed in Nats - this just means they use natural lograithms in cross-entropy calculation

<img src="img/learning2.png" width=450>

## Optimal frontier
If we send <u>more</u> data than optimal, we encounter <u>underfit</u><br>the model have not enough time to update its weights properly; we are higher than the forntier line

If we send <u>less</u> data than optimal, we encounter <u>overfit</u><br>
; we are higher than the forntier line

Если подать больше данных, чем оптимально => underfit
Если подать меньше данных, чем оптимально => overfit





Предположим, 
1) у нас есть оптимальная обученная модель
2) надо её обновить/улучшить и на следующий месяц нам выделили повышенную квоту на compute => можем обучить еще раз
Что выгоднее: увеличить размер модели и обучать столько же или размер оставить и обучать дольше?

Ответ авторов очевиден - важнее размер модели, при этом обучать надо тоже дольше (но меньше).

Что такое оптимально обученая LLM модель?
это модель при заданном лимите на Compute (ND) достигающая наибольшей точности

Авторы замеряют только pre-training LLM модели, fine-tuning или reinforcement learning не цчитываются


В 2022 авторы Chinchilla показали, что Kaplan были не совсем правы и недооценивали важность D
Причина была в том, что они не проверяли на действительно больших объемах, где закономерности немного меняются
К тому же показали, что GPT-3 была недообучна (можно было её улучшить)

Мы считаем, что ресурсы используются эффективно - то есть Compute распределен между N и D так, чтобы получить наилучший performance. А именно:
$$N = D^{0.67}$4

Откуда соотношение N и D?

Теория
Зная вид функции потерь, можно подставить произвольную изокосту и найти минимум на этой кривой. Он довольно просто выражается
$$D = k N^{\gamma=\frac{\alpha}{\beta}}$$
$\alpha$ и $\beta$ ранее также оценивались $\alpha=0.076$, $\beta=1.25$, получаем формулу
Вывод чуть сложнее, чем финальное выражение
Выберем C_0 и выпишем L

Практика
Однако, конкретное значение они подбирали эмпирически

## Overview of laws
Here is the list of all described laws

<img src="img/kaplan_overview.png" width=750>

# Architecture tweaks

Approach: they fix total number of parameters, dataset size and vary some aspect of the model

### Feedforward Ratio

Feedforward Ratio = inner dimensionality of the second FFN layer (as a rate of input/output layer dimensionality). Varies in [0.5, 10]

How FFN shape affects performance

<img src="img/arch1.png" width=300>

Increasing dimensions in FFN means they have to deduct weights from somewhere else to maintain the overall amount. 
They deduct from Attention layer

Two strategies how to do it:
- number of attention heads are fixed, but their dimensionality varies
- dimensionality fixed, but the number of heads varies

Conclusion: it is better to make FFN thinner

*all evaluations were run on rather small models

__Aspect Ratio__

Aspect Ratio = model dimensionality /  model layers

<img src="img/arch2.png" width=300>

There is clearly some in-between balance. Too wide and too narrow networks make performance degrade. But not too much (2-4%) - even poor model can be trained to perform well if there is enough compute

Conclusion: dimensionality should be 10-100 times larger than number of layers

Evaluations were run at three scales, the pattern persists

### LSTM comparison

Transformers are comparable with LSTMs for small contexts, but win a lot for larger ones
per-token loss = an averaged loss when predicting the next token on a position P

<img src="img/lstm1.png" width=350>

<img src="img/lstm2.png" width=350>

### Including embeddings
Почему они не включают эмбединги в кол-во параметров N?
В маленьких моделях они могут составлять значительную часть всей модели (5-30%), а польза от них статичная
Графики это иллюстрируют - на них мальенкие модели (100M), добавление слоев к узкой модели сильно ее улучшает. Добавление слоев к нормальной модели не сильно ее улдушает. 
Даже двуслойные модели можно неплохо обучить (удивительно)

<img src="img/arch4.png" width=300>

<img src="img/arch5.png" width=300>





# Examples
---

For a 1B model<br>
$\begin{cases}
  A100: \,\, 1 \text{ Exa-FLOPs per hour} = 10^{18} \text{ FLOPs per hour} \\
  6ND: \,\, 6 * 10^9 * 10^8 \text{ FLOPS required to get a properly trained 1B model}
\end{cases}$

Time = 1600 hours (~6000$)

Kaplan only states that $D \propto N^{0.76}$ $\big( D = k N^{0.76}\big)$ if you need to be efficient<br>The community estimated the concrete training factor k = 20:<br>
$D = 20 \cdot N^{0.76}$

# Chinchilla Scaling (2020)
---
[[paper]](https://arxiv.org/abs/2203.15556)

Two years later DeepMind challenged the conclusions made by OpenAI and made a statement that Kaplan et al significantly <u>underestimated</u> the importance of dataset size D

The correct combination is <br> $D \propto N^{1.02}$

They explain with small model sizes (1B maximum)
Причина была в том, что они не проверяли на действительно больших объемах, где закономерности немного меняются
К тому же показали, что GPT-3 была недообучна (можно было её улучшить)

Мы считаем, что ресурсы используются эффективно - то есть Compute распределен между N и D так, чтобы получить наилучший performance. А именно:
$$N = D^{0.67}$$

### Approach 1
- Pre-build a loss surface
- For a chosen C make a slice and return optimal value
### Approach 2
- For a chosen C loop through the constraint and fill with model loss
- Return optimal value
### Apporach 3
- Find an analytical solution

Hypothetical example:

<img src='img/syntetic_chinchilla.png' width=500>